# Automated Photonics Test Setup - MZI Phase Shifter

This Jupyter notebook is part of a modular test framework for photonics experiments. It orchestrates instrument initialization, laser sweeps, and data acquisition, using a structured configuration system.

## 1. Overview

This notebook supports an experimental photonics setup by integrating device configuration and data acquisition. Instrument control and measurement parameters are managed externally via a YAML configuration file.

The goal is to automate routine measurements using defined sweep protocols for laser and source meter instruments, ensuring reproducibility and traceability of test results.

## 2. Dependencies
Modules imported from the project:
- Instrument drivers for Agilent8163 and Keithley2400
- Sweep control functions for laser and source meter
- Utilities for config parsing, plotting, and data saving

These modules abstract SCPI communication and enable experiment control through high-level Python functions.

In [ ]:
import pyvisa as visa
from instruments.agilent8163 import Agilent8163Multimeter
from instruments.keithley2400 import Keithley2400SourceMeter
from controllers.laser_sweep import *
from controllers.sourcemeter_sweep import measure_iv_curve, measure_liv_curve
from utils.load_config import load_config
from utils.plotter import plot_measurements
from utils.data_saver import save_raw_measurements

## 3. Configuration Parameters
The experiment configuration is loaded from a YAML file to enable flexible and clean parameter management:

In [ ]:
config = load_config('configs/experiment_config.yaml')


This file defines:
- Agilent laser/meter parameters
- Keithley sourcemeter settings
- Sweep range and step size

In [ ]:
laser_cfg = config["Agilent8163Multimeter"]                         # Extract laser-specific configuration
keithley_cfg = config["Keithley2400"]                               # Extract Keithley instrument configuration
combined_name = f"{config['DUT']['type']}_{config['DUT']['name']}"  # Combined name for saving data

This allows quick changes to experimental parameters without modifying the code.

## 4. Instrument Initialization
Initialize the instruments using configuration values:

In [ ]:
# They are connected on the same GPIB bus, so we need to pass the same resource manager to both instruments
# If you are using multiple GPIB buses, then you need to create separate resource managers for each bus
rm = visa.ResourceManager()

# Generate the laser and keithley objects using the configuration
laser_obj = Agilent8163Multimeter(
    address=laser_cfg['address'],
    laser_slot=laser_cfg['laser_slot'],
    power_slot= laser_cfg['power_slot'],
    power_channel=laser_cfg['power_channel'],
    resource_manager=rm
)

keithley_obj = Keithley2400SourceMeter(address=keithley_cfg['address'], resource_manager=rm)

This instantiates and configures the instruments based on the external YAML file. Each object will handle its own SCPI communication logic internally.

## 5. Measurement Routines
We support the following core experiments:
- Laser Sweep: Varying laser wavelength and recording power output.
- IV Sweep: Varying voltage or current and measuring resulting current or voltage.
- LIV Sweep: Combines laser and IV control for optoelectronic device characterization.

Each routine loads settings from the config and logs raw data to disk.

In [ ]:
laser_sweep_params = {
  "laser": laser_obj,
  "start_wl": laser_cfg["start_wavelength"],
  "stop_wl": laser_cfg["stop_wavelength"],
  "step": laser_cfg["step"],
  "delay": laser_cfg["delay"],
  "logger": None,
}

headers_laser_sweep, results_laser_sweep = perform_laser_sweep(**laser_sweep_params)

# Get the maximum power and its corresponding wavelength
wavelengths, powers = np.array(results_laser_sweep).T
max_idx = powers.argmax()
mzi_peak = powers[max_idx]
mzi_peak_wavelength = wavelengths[max_idx]
print(f"Peak power: {mzi_peak} at wavelength: {mzi_peak_wavelength} nm")


In [ ]:
resistor_measurement_params = {
    "resistance_range": keithley_cfg["resistance_range"],
    "mode": "MAN",
    "offset_comp": "OFF",
    "voltage_prot": keithley_cfg["compliance_voltage"],
    "current_prot": keithley_cfg["compliance_current"],
    "source_func": "CURR",
    "source_level": 0.001,  # The units depend on the source_functions
    "wire_mode": keithley_cfg["wire_mode"],
}

resistance = keithley_obj.read_resistance_configured(**resistor_measurement_params)
print(f"Measured resistance: {resistance} Ohm")


In [ ]:
iv_sweep_params = {
    "sourcemeter": keithley_obj,
    "start_v": keithley_cfg["start_voltage"],
    "stop_v": keithley_cfg["stop_voltage"],
    "step": keithley_cfg["step_voltage"],
    "measure_current_range": keithley_cfg["measure_current_range"],
    "current_limit": keithley_cfg["compliance_current"],
    "wire_mode": keithley_cfg["wire_mode"],
    "delay": keithley_cfg["delay"],
    "logger": None,
}

headers_iv_sweep, results_iv_sweep = measure_iv_curve(**iv_sweep_params)

In [ ]:
liv_params = {
    "sourcemeter": keithley_obj,
    "powermeter": laser_obj,
    "laser": laser_obj,
    "start_v": keithley_cfg["start_voltage"],
    "stop_v": keithley_cfg["stop_voltage"],
    "step": keithley_cfg["step_voltage"],
    "measure_current_range": keithley_cfg["measure_current_range"],
    "current_limit": keithley_cfg["compliance_current"],
    "wire_mode": keithley_cfg["wire_mode"],
    "center_wavelength": mzi_peak_wavelength,
    "sourcemeter_delay": keithley_cfg["delay"],
    "powermeter_delay": laser_cfg["delay"],
    "logger": None,
}

headers_liv_sweep, results_liv_sweep = measure_liv_curve(**liv_params)

## 6. Results Visualization

Measurement results are visualized using custom plotters. Example outputs include:
- Power vs. Wavelength
- IV curve (Current vs. Voltage)
- LIV characteristics (Light Output vs. Current)
- All plots are saved in the plots/ directory and labeled with relevant parameters.

In [ ]:
# Plot the laser sweep results
plot_path = plot_measurements(headers=headers_laser_sweep, 
           results= results_laser_sweep, 
           figure_name=combined_name + "_Laser_Sweep", 
           show=True,
           )

In [ ]:
# Plot the IV sweep results
plot_path = plot_measurements(headers=headers_iv_sweep, 
           results= results_iv_sweep, 
           figure_name=combined_name + "_Sourcemeter_Sweep_IV", 
           show=True,
           )

In [ ]:
# Plots the LIV Resukts
# Trim headers (drop voltage)
trimmed_headers = (headers_liv_sweep[0],headers_liv_sweep[2])  # -> ("Voltage (V)", "Optical Power (dBm)")
# Trim data
trimmed_results = [(v, l) for v, _, l in results_liv_sweep]  # -> List[Tuple[float, float]]

plot_path = plot_measurements(
    headers=trimmed_headers,
    results=[res[:2] for res in trimmed_results],
    figure_name=combined_name + "_Sourcemeter_Sweep_LIV_results", 
    show=True
)

In [ ]:
# Calculate the dips from the results. Use scipy find_peaks
from scipy.signal import find_peaks
dips = -np.array([power for voltage, power in trimmed_results])
peaks, properties = find_peaks(dips, prominence=1)
print(f"Dips found at indices: {peaks}")
for peak in peaks:
    voltage, optical_power = trimmed_results[peak]
    print(f"At index {peak}: Voltage = {voltage:.2f} V, Optical Power = {optical_power} dBm")
    
# Power Consumption
print(f"Total Resistance: {resistance} Ohms")
voltage,_ = trimmed_results[peaks[0]]  # Get the first peak
power_consumption = (voltage**2 / resistance[0])*1e3 # Convert to mW
print(f"Power Consumption at first dip: {power_consumption:.4f} mW")


## 7. Logging and Data Management
Raw measurement data is saved in the data/raw/ folder in CSV format. The utils/logger.py utility ensures each experiment run is timestamped for traceability.

In [ ]:
# Save the laser sweep results to a CSV file
data_path = save_raw_measurements(headers=headers_laser_sweep,
                      data=results_laser_sweep,
                      filename=combined_name + "_Laser_Sweep",
)

In [ ]:
# Save the laser sweep results to a CSV file
data_path = save_raw_measurements(headers=headers_iv_sweep,
                      data=results_iv_sweep,
                      filename=combined_name + "_Sourcemeter_Sweep_IV",
)

In [ ]:
# Save the laser sweep results to a CSV file
data_path = save_raw_measurements(headers=headers_liv_sweep,
                      data=results_liv_sweep,
                      filename=combined_name + "_Sourcemeter_Sweep_LIV_results",
)

## 8. Instrument Shutdown

To properly close the VISA sessions and safely disconnect from the instruments:



In [ ]:
keithley_obj.close()
laser_obj.close()

## 9. Final Notes
This modular approach makes it easy to:

- Extend the setup with new instruments
- Adapt to different photonic device tests
- Maintain consistency across multiple sessions and users

Always ensure your configuration is correct before running full sweeps to prevent instrument misuse.